<a href="https://colab.research.google.com/github/ryuzuiin/buildtransformerfromsratch/blob/main/Japanese_NER_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mecab-python3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 8.9 MB/s eta 0:00:00


In [3]:
!pip install unidic-lite

In [4]:
!pip install --no-binary :all: mecab-python3

In [5]:
!pip install fugashi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 4.2 MB/s eta 0:00:00


In [6]:
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 42.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=2a478e79f0b68b276f89fb243eb760c56816207e9cad8c3e76d7925cf759ebaa
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [7]:
!git clone https://github.com/stockmarkteam/ner-wikipedia-dataset

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 35 (delta 11), reused 11 (delta 1), pack-reused 0
Receiving objects: 100% (35/35), 664.15 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (11/11), done.


steps:
1. input the dataset, split train,validation, test set.
2. split the text;
3. tokenize words and set labels
4. embedding names, make them suitable for bert model
5.dataloaders, set the batch_size
6.train the model:
  a.train set forward and feedback;
  b. validation forward
7.test the model
8. REST API

In [8]:
import os
import json
import unicodedata
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [1]:
# label is type_id

dataset = json.load(open('ner-wikipedia-dataset/ner.json','r'))
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

NameError: name 'json' is not defined

In [13]:
dataset[0:10]

[{'curid': '3572156',
  'text': 'SPRiNGSと最も仲の良いライバルグループ。',
  'entities': [{'name': 'SPRiNGS', 'span': [0, 7], 'type_id': 4}]},
 {'curid': '2415078',
  'text': 'レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。',
  'entities': [{'name': 'レッドフォックス株式会社', 'span': [0, 12], 'type_id': 2},
   {'name': '東京都千代田区', 'span': [14, 21], 'type_id': 5}]},
 {'curid': '2470626',
  'text': '松友美佐紀は、日本のバドミントン選手。',
  'entities': [{'name': '松友美佐紀', 'span': [0, 5], 'type_id': 1},
   {'name': '日本', 'span': [7, 9], 'type_id': 5}]},
 {'curid': '2415752',
  'text': 'ライターの兵庫慎司は普通にアイドルポップスとして出すと売れず、無理にバンドとコラボレーションさせるのも先例からして上手くいかない、それならロックミュージシャンと制作すればいいということになったのではないかとしている。',
  'entities': [{'name': '兵庫慎司', 'span': [5, 9], 'type_id': 1}]},
 {'curid': '2312857',
  'text': 'またCMソング、コミックスイメージアルバム、ゲームミュージック等にボーカルとして多数参加している他、コーラスとして他のアーティストのアルバム等にも参加している。',
  'entities': []},
 {'curid': '159982',
  'text': '2019年現在、日本の航空会社では日本航空、全日本空輸、AIRDOの3社で使用されている。',
  'entities': [{'name': '日本', 'span': [8, 10], 'type_id': 5},
   {'name': 

In [3]:
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

NameError: name 'dataset' is not defined

In [17]:
dataset[0:10]

[{'curid': '3572156',
  'text': 'SPRiNGSと最も仲の良いライバルグループ。',
  'entities': [{'name': 'SPRiNGS', 'span': [0, 7], 'type_id': 4}]},
 {'curid': '2415078',
  'text': 'レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。',
  'entities': [{'name': 'レッドフォックス株式会社', 'span': [0, 12], 'type_id': 2},
   {'name': '東京都千代田区', 'span': [14, 21], 'type_id': 5}]},
 {'curid': '2470626',
  'text': '松友美佐紀は、日本のバドミントン選手。',
  'entities': [{'name': '松友美佐紀', 'span': [0, 5], 'type_id': 1},
   {'name': '日本', 'span': [7, 9], 'type_id': 5}]},
 {'curid': '2415752',
  'text': 'ライターの兵庫慎司は普通にアイドルポップスとして出すと売れず、無理にバンドとコラボレーションさせるのも先例からして上手くいかない、それならロックミュージシャンと制作すればいいということになったのではないかとしている。',
  'entities': [{'name': '兵庫慎司', 'span': [5, 9], 'type_id': 1}]},
 {'curid': '2312857',
  'text': 'またCMソング、コミックスイメージアルバム、ゲームミュージック等にボーカルとして多数参加している他、コーラスとして他のアーティストのアルバム等にも参加している。',
  'entities': []},
 {'curid': '159982',
  'text': '2019年現在、日本の航空会社では日本航空、全日本空輸、AIRDOの3社で使用されている。',
  'entities': [{'name': '日本', 'span': [8, 10], 'type_id': 5},
   {'name': 

In [19]:
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

print(f"Length of train: {len(dataset_train)}")
print(f"Length of val: {len(dataset_val)}")
print(f"Length of test: {len(dataset_test)}")

Length of train: 3205
Length of val: 1068
Length of test: 1070


In [11]:
#details of BertJapaneseTokenizer is here: https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert_japanese/tokenization_bert_japanese.py?source=post_page-----7b54120aa245--------------------------------

class NerTokenizerForTrain(BertJapaneseTokenizer):

  def create_tokens_and_labels(self, splitted):
      """分割された文字列をトークン化し、ラベルを付与
      Args：
        splitted: 分割された文字列
          例：
          [{'text': 'レッドフォックス株式会社', 'label': 2},
          {'text': 'は、', 'label': 0},
          {'text': '東京都千代田区', 'label': 5},
          {'text': 'に本社を置くITサービス企業である。', 'label': 0}]
      Return:
        tokens, labels
          例：
          ['レッド', 'フォックス', '株式会社', 'は', '、', '東京', '都', '千代田', '区', 'に', '本社', 'を', '置く', 'IT', 'サービス', '企業', 'で', 'ある', '。']
          [2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      """
      tokens = [] # トークン格納用
      labels = [] # トークンに対応するラベル格納用
      for s in splitted:
          text = s['text']
          label = s['label']
          tokens_splitted = self.tokenize(text) # BertJapaneseTokenizerのトークナイザを使ってトークンに分割
          labels_splitted = [label] * len(tokens_splitted)
          tokens.extend(tokens_splitted)
          labels.extend(labels_splitted)

      return tokens, labels


  def encoding_for_bert(self, tokens, labels, max_length):
      """符号化を行いBERTに入力できる形式にする
      Args:
        tokens: トークン列
        labels: トークンに対応するラベルの列
      Returns:
        encoding: BERTに入力できる形式
        例：
        {'input_ids': [2, 3990, 13779, 1275, 9, 6, 391, 409, 9674, 280, 7, 2557, 11, 3045, 8267, 1645, 1189, 12, 31, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        　'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        　'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
          'labels': [0, 2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

      """
      encoding = self.encode_plus(
          tokens,
          max_length=max_length,
          padding='max_length',
          truncation=True
      )
      # トークン[CLS]、[SEP]のラベルを0
      labels = [0] + labels[:max_length-2] + [0]
      # トークン[PAD]のラベルを0
      labels = labels + [0]*( max_length - len(labels) )
      encoding['labels'] = labels

      return encoding


  def encode_plus_tagged(self, text, entities, max_length):
      """文章とそれに含まれる固有表現が与えられた時に、符号化とラベル列の作成
      Args:
        text: 元の文章
        entities: 文章中の固有表現の位置(span)とラベル(type_id)の情報

      """
      # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
      entities = sorted(entities, key=lambda x: x['span'][0]) # 固有表現の位置の昇順でソート
      splitted = [] # 分割後の文字列格納用
      position = 0
      for entity in entities:
          start = entity['span'][0]
          end = entity['span'][1]
          label = entity['type_id']
          # 固有表現ではないものには0のラベルを付与
          splitted.append({'text': text[position:start], 'label':0})
          # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
          splitted.append({'text': text[start:end], 'label':label})
          position = end

      # 最後の固有表現から文末に、0のラベルを付与
      splitted.append({'text': text[position:], 'label':0})
      # positionとspan[0]の値が同じだと空白文字にラベル0が付与されるため、長さ0の文字列は除く（例：{'text': '', 'label': 0}）
      splitted = [ s for s in splitted if s['text'] ]

      # 分割された文字列をトークン化し、ラベルを付与
      tokens, labels = self.create_tokens_and_labels(splitted)

      # 符号化を行いBERTに入力できる形式にする
      encoding = self.encoding_for_bert(tokens, labels, max_length)

      return encoding


In [12]:
tokenizer = NerTokenizerForTrain.from_pretrained(MODEL_NAME)
import pprint
tmp = dataset_train[1]
pprint.pprint(tmp)
pprint.pprint(tokenizer.encode_plus_tagged(text=tmp["text"], entities=tmp["entities"], max_length=32), width=200)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NerTokenizerForTrain'.


{'curid': '2415078',
 'entities': [{'name': 'レッドフォックス株式会社', 'span': [0, 12], 'type_id': 2},
              {'name': '東京都千代田区', 'span': [14, 21], 'type_id': 5}],
 'text': 'レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。'}
{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [2, 3990, 13779, 1275, 9, 6, 391, 409, 9674, 280, 7, 2557, 11, 3045, 8267, 1645, 1189, 12, 31, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'labels': [0, 2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [13]:
class CreateDataset(Dataset):
  """データセット作成
  """
  def __init__(self, dataset, tokenizer, max_length):
    self.dataset = dataset
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    text = self.dataset[index]["text"]
    entities = self.dataset[index]["entities"]
    encoding = tokenizer.encode_plus_tagged(text, entities, max_length=self.max_length)

    input_ids = torch.tensor(encoding["input_ids"])
    token_type_ids = torch.tensor(encoding["token_type_ids"])
    attention_mask = torch.tensor(encoding["attention_mask"])
    labels = torch.tensor(encoding["labels"])

    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_mask,
      "labels": labels
    }


In [14]:
# データセットの作成
dataset_train_for_loader = CreateDataset(dataset_train, tokenizer, max_length=128)
dataset_val_for_loader = CreateDataset(dataset_val, tokenizer, max_length=128)

# データローダーの作成
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=32, shuffle=True, pin_memory=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256, shuffle=True, pin_memory=True)

dataloaders_dict = {"train": dataloader_train, "val": dataloader_val}


In [15]:
# GPU使えるならGPU使う
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 学習済みモデルのロード
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=9)

# モデルをGPUへ転送
model.to(device)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [16]:
# 最適化器
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)


In [17]:
def train_model(net, dataloaders_dict, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    loss, logits = model(input_ids=input_ids,
                                          token_type_ids=None,
                                          attention_mask=attention_mask,
                                          labels=labels,
                                          return_dict=False)

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            print(f"イテレーション {iteration} || Loss: {loss:.4f}")

                    iteration += 1

                    # 損失の合計を更新
                    epoch_loss += loss.item() * batch_size

            # epochごとのloss
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print(f"Epoch {epoch+1}/{num_epochs} | phase {phase} |  Loss: {epoch_loss:.4f}")

    return net


In [18]:
# 学習・検証を実行
num_epochs = 5
net_trained = train_model(model, dataloaders_dict, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 0.4222
イテレーション 20 || Loss: 0.2417
イテレーション 30 || Loss: 0.1731
イテレーション 40 || Loss: 0.1725
イテレーション 50 || Loss: 0.0953
イテレーション 60 || Loss: 0.0731
イテレーション 70 || Loss: 0.0394
イテレーション 80 || Loss: 0.0493
イテレーション 90 || Loss: 0.0634
イテレーション 100 || Loss: 0.0389
Epoch 1/5 | phase train |  Loss: 0.2206
Epoch 1/5 | phase val |  Loss: 0.0052
イテレーション 10 || Loss: 0.0273
イテレーション 20 || Loss: 0.0216
イテレーション 30 || Loss: 0.0228
イテレーション 40 || Loss: 0.0150
イテレーション 50 || Loss: 0.0310
イテレーション 60 || Loss: 0.0468
イテレーション 70 || Loss: 0.0258
イテレーション 80 || Loss: 0.0241
イテレーション 90 || Loss: 0.0244
イテレーション 100 || Loss: 0.0094
Epoch 2/5 | phase train |  Loss: 0.0247
Epoch 2/5 | phase val |  Loss: 0.0033
イテレーション 10 || Loss: 0.0097
イテレーション 20 || Loss: 0.0184
イテレーション 30 || Loss: 0.0254
イテレーション 40 || Loss: 0.0108
イテレーション 50 || Loss: 0.0103
イテレーション 60 || Loss: 0.0191
イテレーション 70 || Loss: 0.0116
イテレーション 80 || Loss: 0.0054
イテレーション 90 || Loss: 0.0100
イテレーション 100 || Loss: 0.013

In [19]:
class NerTokenizerForTest(BertJapaneseTokenizer):

    def encoding_for_bert(self, tokens, max_length):
        """符号化を行いBERTに入力できる形式にする
        Args:
          tokens: トークン列
        Returns:
          encoding: BERTに入力できる形式
          例：
          {'input_ids': [2, 106, 6, 946, 674, 5, 12470, 9921, 5, 859, 6, 2446, 22903, 35, 24831, 11614, 35, 2176, 2200, 35, 3700, 29650, 2446, 333, 9, 6, 2409, 109, 5, 333, 3849, 3],
          'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        """
        encoding = self.encode_plus(
            tokens,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors = "pt"
        )

        return encoding


    def create_spans_of_token(self, tokens_original, encoding):
        """ 各トークン（サブワード）の文章中での位置を調べる
          Args:
            tokens_original: トークン列をさらにサブワードに分割した列
            encoding:
            例：tokens_original
              ['元々', 'は', '前作', '「', 'The', 'Apple', 's', '」', 'の', 'アウト', ...]

          Return:
            spans: 各トークンの文章中の位置([CLS][PAD]などの特殊トークンはダミーで置き換える)
            例：
              [[-1, -1], [0, 2], [2, 3], [3, 5], [5, 6], [6, 9], [10, 15], [15, 16], ...]
        """
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    """例：英語文章のように空白が混ざっていると下記のようにずれるケースがあることを考慮
                          token: "Digital"
                          text[position:position+l]: " Digita"
                    """
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2]
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) )

        return spans


    def encode_plus_untagged(self, text, max_length=None):
        """文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークン格納用
        tokens_original = [] # トークンに対応する文章中の文字列格納用
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])


        # 符号化を行いBERTに入力できる形式にする
        encoding = self.encoding_for_bert(tokens, max_length)

        # 各トークン（サブワード）の文章中での位置を調べる
        spans = self.create_spans_of_token(tokens_original, encoding)

        return encoding, spans


    def convert_bert_output_to_entities(self, text, labels, spans):
        """文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        position = 0
        for label, group in itertools.groupby(labels):
            """
            例：labelsは予測結果
            labels: [0, 0, 0, 3, 3, 5, 7, 7, 7, 0, 0, 0]
            """
            start_idx = position # 連続するラベルの先頭位置
            end_idx = position + len(list(group)) - 1 # 連続するラベルの最終位置

            # (encode_plus_untaggedで計算した)spansから、文章中の位置を特定
            start = spans[start_idx][0]
            end = spans[end_idx][1]

            # 次のspanの位置に更新
            position = end_idx + 1

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities


In [20]:
# テスト時に使うトークナイザーをロード
tokenizer = NerTokenizerForTest.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NerTokenizerForTest'.


In [21]:
def predict(text, tokenizer, model):
    """BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(text)
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = model(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

# 固有表現抽出
entities_list = [] # 正解の固有表現
entities_predicted_list = [] # 予測された固有表現
for sample in tqdm(dataset_test):
    text = sample['text']
    entities_predicted = predict(text, tokenizer, net_trained) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

100%|██████████| 1070/1070 [00:39<00:00, 27.28it/s]


In [22]:
entities_list = [] # 正解の固有表現
entities_predicted_list = [] # 予測された固有表現
for sample in tqdm(dataset_test):
    text = sample['text']
    entities_predicted = predict(text, tokenizer, net_trained) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

100%|██████████| 1070/1070 [00:39<00:00, 27.31it/s]


In [23]:
i = 1
print("# 正解 #")
print(entities_list[i])
print("# 推論 #")
print(entities_predicted_list[i])
print("# もとの文章 #")
print(dataset_test[i]["text"])


# 正解 #
[{'name': 'セカンダリースクール', 'span': [0, 10], 'type_id': 2}, {'name': '南アフリカ高等弁務局', 'span': [24, 34], 'type_id': 3}]
# 推論 #
[{'name': 'セカンダリースクール', 'span': [0, 10], 'type_id': 6}, {'name': '南アフリカ高等弁務局', 'span': [24, 34], 'type_id': 3}]
# もとの文章 #
セカンダリースクールを卒業後、ホテルなどで働いた南アフリカ高等弁務局の秘書官として従事した折に政治への興味を持ったとされる。


In [24]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [
                e for e in entities_predicted if e['type_id'] == type_id
            ]

        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [25]:
# 評価結果
import pandas as pd
eval_df = pd.DataFrame()
for k, v in type_id_dict.items():
  eval_res = evaluate_model(entities_list, entities_predicted_list, type_id=v)
  eval_df[k] = eval_res.values()

eval_res_all = evaluate_model(entities_list, entities_predicted_list, type_id=None)
eval_df["ALL"] = eval_res_all.values()

eval_df.index = eval_res_all.keys()
eval_df

,人名,法人名,政治的組織名,その他の組織名,地名,施設名,製品名,イベント名,ALL
num_entities,604.000000,504.000000,249.000000,222.000000,452.000000,222.000000,231.000000,203.000000,2687.000000
num_predictions,637.000000,510.000000,270.000000,218.000000,453.000000,249.000000,244.000000,193.000000,2774.000000
num_correct,588.000000,457.000000,215.000000,184.000000,394.000000,188.000000,177.000000,167.000000,2370.000000
precision,0.923077,0.896078,0.796296,0.844037,0.869757,0.755020,0.725410,0.865285,0.854362
recall,0.973510,0.906746,0.863454,0.828829,0.871681,0.846847,0.766234,0.822660,0.882025
f_value,0.947623,0.901381,0.828516,0.836364,0.870718,0.798301,0.745263,0.843434,0.867973
